<a href="https://colab.research.google.com/github/akshayasanghavi-commits/Learningpython1/blob/main/AgenticResumeTailor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Creating a AgenticAI which tailors resume as per job discription with fact check Agnet and RAG
#!pip install -q crewai langchain-google-genai 'crewai[tools]'
!pip install pypdf
import os
from crewai import Agent, Task, Crew
from crewai_tools import PDFSearchTool
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from pypdf import PdfReader
# 1. Setup
#os.environ["OPENAI_API_KEY"] = "NA"
os.environ["GOOGLE_API_KEY"] = userdata.get('GoogleKey')
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
print(os.path.exists('/content/AK_CV.pdf')) # Should return True
#pdf_tool = PDFSearchTool( pdf='/content/AK_CV.pdf')

def get_pdf_text(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        content = page.extract_text()
        if content:
            text += content + "\n"
    return text

# Get your resume text
resume_text = get_pdf_text('/content/AK_CV.pdf')

# 2. INPUT: Paste the Job Description here
job_description = """
We are looking for a Python Developer with experience in AI Agents,
Pandas, and building RAG systems. Knowledge of API security is a plus.
"""

# 3. Define the Agents
tailor = Agent(
    role='Resume Strategist',
    goal=f'Identify missing keywords from the JD and suggest updates to the resume summary.and resume conetnt text is{resume_text} ',
    backstory='You are an expert at matching candidate skills to specific job requirements.',
    #tools=[pdf_tool], # Tailor reads the resume to see what we already have
    llm=llm
)

fact_checker = Agent(
    role='Honesty Auditor',
    goal=f'Ensure all suggested updates are strictly based on actual facts in the original resume and resume conetnt text is{resume_text}.',
    backstory='You are a strict auditor who prevents resume inflation or lying.',
   # tools=[pdf_tool], # Fact-checker verifies against the source
    llm=llm
)

# 4. Define the Tasks
task_tailor = Task(
    description=f"""
    1. Compare the uploaded resume to this Job Description: {job_description}
    2. Write a new 3-sentence summary for the resume that highlights
       the specific skills the employer is looking for.
    """,
    expected_output="A tailored 3-sentence professional summary.",
    agent=tailor
)

task_verify = Task(
    description="""
    Review the tailored summary. Cross-reference every claim with the original resume content.
    If the summary mentions a skill NOT found in the PDF, remove it or rephrase it.
    """,
    expected_output="A fact-checked, honest, and tailored resume summary.",
    agent=fact_checker,
    context=[task_tailor]
)

# 5. Run the Crew
crew = Crew(agents=[tailor, fact_checker], tasks=[task_tailor, task_verify])
result = crew.kickoff()

print("\n--- FINAL TAILORED & VERIFIED SUMMARY ---")
print(result)


True

--- FINAL TAILORED & VERIFIED SUMMARY ---
Highly motivated Python Developer with specialized expertise in AI Agents and robust data manipulation using Pandas, re-entering the tech sector with a strong foundation in modern AI. Proven ability to implement agentic AI solutions, as demonstrated by projects utilizing CrewAI and LLMs for advanced automation. Adept at leveraging Python for data analysis, machine learning model development, and eager to apply these skills to build sophisticated AI systems.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful